In [ ]:
import matplotlib
from matplotlib import pyplot
matplotlib.interactive(False)
matplotlib.use('Agg')
import os
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing melspec

In [ ]:
corepath = '/content/drive/MyDrive/DL/data/'
melsPath= '/content/drive/MyDrive/DL/mels/'

In [ ]:
birds = []  # list of all birds
for root, dirs, files in os.walk(corepath):
    if root == corepath:
        birds = dirs

birds50 = []
flist = []  # list of all files
blist = []  # list of files for one bird
i50 = 0
max_files = 65  # batas maksimal file yang diambil

for i, bird in enumerate(birds):
    for root, dirs, files in os.walk(corepath + bird):
        for file in files:
            if file.endswith(".wav"):
                blist.append(os.path.join(root, file))
                # Cek jika jumlah file dalam blist sudah mencapai batas
                if len(blist) >= max_files:
                    break  # keluar dari loop jika sudah mencapai batas
        if len(blist) > 50:  # hanya ambil jika lebih dari 50 file
            i50 += 1
            print(i50, ". Found ", len(blist), ' files for ', bird, '(', i + 1, ')')
            birds50.append(bird)
            flist.append(blist)
        blist = []  # reset blist untuk bird berikutnya

1 . Found  65  files for  Pitta sordida ( 1 )
2 . Found  65  files for  Dryocopus javensis ( 2 )
3 . Found  65  files for  Caprimulgus macrurus ( 3 )
4 . Found  65  files for  Pnoepyga pusilla ( 4 )
5 . Found  65  files for  Anthipes solitaris ( 5 )
6 . Found  65  files for  Buceros rhinoceros ( 6 )


In [ ]:
def saveMel(signal, directory):
    gc.enable()
    # MK_spectrogram modified

    n_fft = 1024
    hop_size = 512
    n_mels = 128
    win_size = 512
    window_type = 'hann'
    feature = 'mel'
    fmin = 1400

    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    S = librosa.feature.melspectrogram(y=signal, sr=sr,
                                        n_fft=n_fft,
                                        hop_length=hop_size,
                                        n_mels=n_mels,
                                        htk=True,
                                        fmin=fmin, # higher limit ##high-pass filter freq.
                                        fmax=sr/2) # AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=fmin) #power = S**2

    fig.savefig(directory)
    plt.ioff()
    #plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from tqdm import tqdm_notebook as tqdm
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

size = {'desired': 4, # [seconds]
        'minimum': 2, # [seconds]
        'stride' : 0, # [seconds]
        'name': 5 # [number of letters]
       } # stride should not be bigger than desired length

print('Number of directories to check and cut: ', len(flist))

#step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds
step=1
if step>0:
    for bird, birdList in enumerate(flist):
        print("Processing ",bird,'. ', birds50[bird], "...")
        for birdnr, path in tqdm(enumerate(birdList)):
            # load the mp3 file
            directory=melsPath+str(bird)+birds50[bird][:size['name']]+"/"

            if not os.path.exists(directory):
                os.makedirs(directory)

            if not os.path.exists(directory+path.rsplit('/',1)[1].replace(' ', '')[:-4]+"_1.png"):

                signal, sr = librosa.load(path, sr=48000) # sr = sampling rate
                step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds

                nr=0;
                for start, end in zip(range(0,len(signal),step),range(size['desired']*sr,len(signal),step)):
                    # cut file and save each piece
                    nr=nr+1
                    # save the file if its length is higher than minimum
                    if end-start > size['minimum']*sr:
                        melpath=path.rsplit('/',1)[1]
                        melpath=directory+melpath.replace(' ', '')[:-4]+str(nr)+"_"+str(nr)+".png"
                        saveMel(signal[start:end],melpath)

            pass
else:
    print("Error: Stride should be lower than desired length.")

Number of directories to check and cut:  6
Processing  0 .  Pitta sordida ...


0it [00:00, ?it/s]

Processing  1 .  Dryocopus javensis ...


0it [00:00, ?it/s]

Processing  2 .  Caprimulgus macrurus ...


...

0it [00:00, ?it/s]

Processing  4 .  Anthipes solitaris ...


0it [00:00, ?it/s]

Processing  5 .  Buceros rhinoceros ...


0it [00:00, ?it/s]

In [ ]:
glist=[]
for root, dirs, files in os.walk(melsPath):
    for file in files:
        if file.endswith(".png"):
            glist.append(os.path.join(root, file))

print('Found ', len(glist),' files.')

Found  4840  files.


# Preprocessing mfcc

In [ ]:
mfccPath= '/content/drive/MyDrive/DL/mfcc/'

In [ ]:
def saveMFCC(signal, directory):
    gc.enable()
    n_fft = 1024         #
    hop_size = 512      #
    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    MFFCS = librosa.feature.mfcc(y=signal, sr=sr,
                                        n_fft=n_fft,
                                        hop_length=hop_size,
                                        n_mfcc=40,
                                        htk=True)
    librosa.display.specshow(MFFCS, sr=sr, hop_length=hop_size)

    fig.savefig(directory)
    plt.ioff()
    #plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from tqdm import tqdm_notebook as tqdm
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

size = {'desired': 4, # [seconds]
        'minimum': 2, # [seconds]
        'stride' : 0, # [seconds]
        'name': 5 # [number of letters]
       } # stride should not be bigger than desired length

print('Number of directories to check and cut: ', len(flist))

#step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds
step=1
if step>0:
    for bird, birdList in enumerate(flist):
        print("Processing ",bird,'. ', birds50[bird], "...")
        for birdnr, path in tqdm(enumerate(birdList)):
            # load the mp3 file
            directory=mfccPath+str(bird)+birds50[bird][:size['name']]+"/"

            if not os.path.exists(directory):
                os.makedirs(directory)

            if not os.path.exists(directory+path.rsplit('/',1)[1].replace(' ', '')[:-4]+"_1.png"):

                signal, sr = librosa.load(path, sr=48000) # sr = sampling rate
                step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds

                nr=0;
                for start, end in zip(range(0,len(signal),step),range(size['desired']*sr,len(signal),step)):
                    # cut file and save each piece
                    nr=nr+1
                    # save the file if its length is higher than minimum
                    if end-start > size['minimum']*sr:
                        mfccpath=path.rsplit('/',1)[1]
                        mfccpath=directory+mfccpath.replace(' ', '')[:-4]+str(nr)+"_"+str(nr)+".png"
                        saveMFCC(signal[start:end],mfccpath)
                    #print('New file...',start/sr,' - ',end/sr)
                    #print('Start: ',start,'end: ', end, 'length: ', end-start)

            pass
else:
    print("Error: Stride should be lower than desired length.")

Number of directories to check and cut:  6
Processing  0 .  Pitta sordida ...


0it [00:00, ?it/s]

Processing  1 .  Dryocopus javensis ...


0it [00:00, ?it/s]

Processing  2 .  Caprimulgus macrurus ...


...

0it [00:00, ?it/s]

Processing  4 .  Anthipes solitaris ...


0it [00:00, ?it/s]

Processing  5 .  Buceros rhinoceros ...


0it [00:00, ?it/s]

In [ ]:
clist=[]
for root, dirs, files in os.walk(mfccPath):
    for file in files:
        if file.endswith(".png"):
            clist.append(os.path.join(root, file))

print('Found ', len(clist),' files.')

Found  4840  files.
